# TODO: Title
**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of the TODO's and/or use more than one cell to complete all the tasks.

In [3]:
# TODO: Import any packages that you might need
import sagemaker
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torchvision
import os
import json
import boto3
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
#from sagemaker import get_execution_role

/home/tho121/udacity_aws/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Preparation
**TODO:** Run the cell below to download the data.

The cell below creates a folder called `train_data`, downloads training data and arranges it in subfolders. Each of these subfolders contain images where the number of objects is equal to the name of the folder. For instance, all images in folder `1` has images with 1 object in them. Images are not divided into training, testing or validation sets. If you feel like the number of samples are not enough, you can always download more data (instructions for that can be found [here](https://registry.opendata.aws/amazon-bin-imagery/)). However, we are not acessing you on the accuracy of your final trained model, but how you create your machine learning engineering pipeline.

In [3]:


from tqdm import tqdm

def download_and_arrange_data():
    s3_client = boto3.client('s3')

    with open('large_list.json', 'r') as f:
        d=json.load(f)

    for k, v in d.items():
        print(f"Downloading Images with {k} objects")
        directory=os.path.join('large_train_data', k)
        if not os.path.exists(directory):
            os.makedirs(directory)
        for file_path in tqdm(v):
            file_name=os.path.basename(file_path).split('.')[0]+'.jpg'
            s3_client.download_file('aft-vbi-pds', os.path.join('bin-images', file_name),
                             os.path.join(directory, file_name))

download_and_arrange_data()

100%|██████████| 25987/25987 [38:54<00:00, 11.13it/s] 


100%|██████████| 48103/48103 [1:13:14<00:00, 10.95it/s]


100%|██████████| 56409/56409 [1:27:47<00:00, 10.71it/s] 


100%|██████████| 50384/50384 [1:17:35<00:00, 10.82it/s]


100%|██████████| 39117/39117 [1:01:22<00:00, 10.62it/s]


## Dataset
**TODO:** Explain what dataset you are using for this project. Give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understanding of it. You can find more information about the data [here](https://registry.opendata.aws/amazon-bin-imagery/).

In [3]:
#TODO: Perform any data cleaning or data preprocessing
print(len(os.listdir('large_train_data/1')))
print(len(os.listdir('large_train_data/2')))
print(len(os.listdir('large_train_data/3')))
print(len(os.listdir('large_train_data/4')))
print(len(os.listdir('large_train_data/5')))

1229
2300
2666
2373
1875


In [4]:
dataset = torchvision.datasets.ImageFolder("large_train_data")

In [5]:
test_size = int(0.1 * len(dataset))
train_data, test_data = torch.utils.data.random_split(dataset, [len(dataset) - test_size, test_size])

In [8]:
os.mkdir("resized_train")
os.mkdir("resized_train/1")
os.mkdir("resized_train/2")
os.mkdir("resized_train/3")
os.mkdir("resized_train/4")
os.mkdir("resized_train/5")

In [9]:
os.mkdir("resized_test")
os.mkdir("resized_test/1")
os.mkdir("resized_test/2")
os.mkdir("resized_test/3")
os.mkdir("resized_test/4")
os.mkdir("resized_test/5")

In [8]:
count = 0
for img, label in train_data:
    img.save(os.path.join("large_train", str(label + 1), str(count) +".jpg"))
    count += 1

In [9]:
count = 0
for img, label in test_data:
    img.save(os.path.join("large_test", str(label + 1), str(count) +".jpg"))
    count += 1

In [ ]:
#TODO: Upload the data to AWS S3

s3_path_to_data = sagemaker.Session().upload_data(bucket='bincapstone', 
                                                  path='large_train', 
                                                  key_prefix='capstone/data/large_train')

In [ ]:
s3_path_to_data_test = sagemaker.Session().upload_data(bucket='bincapstone', 
                                                  path='large_test', 
                                                  key_prefix='capstone/data/large_test')

In [48]:
s3_client = boto3.client('s3')
s3_client.download_file('aft-vbi-pds', os.path.join('metadata', "101598.json"),
                             os.path.join("", "101598.json"))

In [38]:
s3 = boto3.resource('s3')
data_bucket = s3.Bucket('aft-vbi-pds')

#arn:aws:s3:::aft-vbi-pds

In [ ]:
import pandas as pd
import json

s3_client = boto3.client('s3')
bucket='aft-vbi-pds'

with open('full_list.json','r+') as file:
    file_data = json.load(file)
    
    
count = 0
for object_summary in data_bucket.objects.filter(Prefix="metadata"):
    key = object_summary.key
    if key.endswith(".json"):
        data_location = 's3://{}/{}'.format(bucket, key)
        
        data = pd.read_json(data_location) 

        num = data['EXPECTED_QUANTITY']
        
        for n in num:
            if n > 0 and n < 6:
                file_data[str(n)].append(key)
                count += 1
                
                if count % 10000 == 0:
                    with open('check.json', 'w+') as file:
                        json.dump(file_data, file)
                    print(count)
            break
        
with open('check.json','w+') as file:
    json.dump(file_data, file)


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000


In [ ]:
import boto3
s3_resource = boto3.client('s3')
paginator = s3_resource.get_paginator('list_objects_v2')
page_iterator = paginator.paginate(Bucket='aft-vbi-pds')
keys=[]
for page in page_iterator:
    for i in page['Contents']:
        keys.append(i['Key'])


In [ ]:
page_iterator[0]

In [ ]:
print(len(keys))

In [ ]:
#Using Boto3 Resource to do the copy
s3_resource = boto3.resource('s3')
bucket = s3_resource.Bucket('s3://bincapstone/capstone/data/all/')
for key in keys:
    copy_source = {
        'Bucket': 'arn:aws:s3:::aft-vbi-pds',
        'Key': key
    }
    
    bucket.copy(copy_source, key)
    
    file_name=os.path.basename(file_path).split('.')[0]+'.jpg'
    s3_client.download_file('aft-vbi-pds', os.path.join('bin-images', file_name), os.path.join(directory, file_name))

In [4]:
import torchvision.transforms as T

transforms = T.Compose([T.Resize((224, 224))])

In [5]:
train_dataset = torchvision.datasets.ImageFolder("large_train", transform=transforms)
test_dataset = torchvision.datasets.ImageFolder("large_test", transform=transforms)

In [11]:
count = 0
for img, label in train_dataset:
    img.save(os.path.join("resized_train", str(label + 1), str(count) +".jpg"))
    count += 1

In [12]:
count = 0
for img, label in test_dataset:
    img.save(os.path.join("resized_test", str(label + 1), str(count) +".jpg"))
    count += 1

In [ ]:
s3_path_to_data = sagemaker.Session().upload_data(bucket='bincapstone', 
                                                  path='resized_train', 
                                                  key_prefix='capstone/data/resized_train')

In [ ]:
s3_path_to_data = sagemaker.Session().upload_data(bucket='bincapstone', 
                                                  path='resized_test', 
                                                  key_prefix='capstone/data/resized_test')

In [23]:
from torch.utils.data import DataLoader

train_data_loader = DataLoader(train_dataset, batch_size=128, shuffle=False, num_workers=1)
test_data_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=1)

In [24]:
def batch_mean_std(loader):
    cnt = 0
    fst_moment = torch.empty(3)
    snd_moment = torch.empty(3)
    
    for images, _ in loader:
        b,c,h,w = images.shape
        nb_pixels = b*h*w
        sum_ = torch.sum(images, dim=[0,2,3])
        sum_of_squares = torch.sum(images **2, dim=[0,2,3])
        
        fst_moment = (cnt * fst_moment + sum_) / (cnt + nb_pixels)
        snd_moment = (cnt * snd_moment + sum_of_squares) / (cnt + nb_pixels)
        
    mean = fst_moment
    std = torch.sqrt(snd_moment - fst_moment ** 2)
    return mean, std
    

In [25]:
mean,std = batch_mean_std(train_data_loader)
print(mean)
print(std)

tensor([0.5300, 0.4495, 0.3624])
tensor([0.1691, 0.1476, 0.1114])


In [26]:
mean,std = batch_mean_std(test_data_loader)
print(mean)
print(std)

tensor([0.5206, 0.4400, 0.3570])
tensor([0.1658, 0.1467, 0.1113])


## Model Training
**TODO:** This is the part where you can train a model. The type or architecture of the model you use is not important. 

**Note:** You will need to use the `train.py` script to train your model.

In [20]:
from sagemaker import get_execution_role
role = get_execution_role()

Couldn't call 'get_role' to get Role ARN from role name Tony to get Role path.


ValueError: The current AWS identity is not a role: arn:aws:iam::149633813601:user/Tony, therefore it cannot be used as a SageMaker execution role

In [32]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='SagemakerFullAccess')['Role']['Arn']

Couldn't call 'get_role' to get Role ARN from role name Tony to get Role path.


In [5]:
#TODO: Declare your model training hyperparameter.
hyperparameters = {'batch_size': 128, 'lr': 0.001}

In [52]:
#TODO: Create your training estimator
estimator = PyTorch(
    entry_point='train.py',
    base_job_name='capstone',
    role=role,
    instance_count=1,
    instance_type='ml.m5.4xlarge',
    framework_version='1.12.0',
    py_version='py38',
    hyperparameters=hyperparameters,
)

In [55]:
# TODO: Fit your estimator
os.environ['SM_CHANNEL_TRAIN']='s3://awscapstone/train_data/'
os.environ['SM_CHANNEL_TEST']='s3://awscapstone/test_data/'
os.environ['SM_MODEL_DIR']='s3://awscapstone/'
estimator.fit({"train": os.environ['SM_CHANNEL_TRAIN'], "test": os.environ['SM_CHANNEL_TEST'] }, wait=True)

2022-12-02 21:59:29 Starting - Starting the training job...
2022-12-02 22:00:00 Starting - Preparing the instances for trainingProfilerReport-1670018374: InProgress
......
2022-12-02 22:01:00 Downloading - Downloading input data...........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-12-02 22:02:48,943 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-12-02 22:02:48,945 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-02 22:02:48,953 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-12-02 22:02:48,959 sagemaker_pytorch_container.training INFO     Invoking SMDataParallel for native PT DDP job
2022-12-02 22:02:48,959 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-12-02 22:02:49,461 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus i

UnexpectedStatusException: Error for Training job capstone-2022-12-02-21-59-33-474: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
ExitCode 134
ErrorMessage ""
Command "mpirun --host algo-1:1,algo-2:1 -np 2 --allow-run-as-root --tag-output --oversubscribe -mca btl_tcp_if_include eth0 -mca oob_tcp_if_include eth0 -mca plm_rsh_no_tree_spawn 1 -mca pml ob1 -mca btl ^openib -mca orte_abort_on_non_zero_status 1 -mca btl_vader_single_copy_mechanism none -mca plm_rsh_num_concurrent 2 -x NCCL_SOCKET_IFNAME=eth0 -x NCCL_DEBUG=INFO -x LD_LIBRARY_PATH -x PATH -x SMDATAPARALLEL_USE_HOMOGENEOUS=1 -x FI_PROVIDER=efa -x RDMAV_FORK_SAFE=1 -x LD_PRELOAD=/opt/conda/lib/python3.8/site-packages/gethostname.cpython-38-x86_64-linux-gnu.so -x SMDATAPARALLEL_SERVER_ADDR=algo-1 -x SMDATAPARALLEL_SERVER_PORT=7592 -x SAGEMAKER_INSTANCE_TYPE=ml.m5.4xlarge smddprun /opt/conda/bin/python3.8 -m mpi4py train.py --batch_size 128 --lr 0.001", exit code: 1

## Standout Suggestions
You do not need to perform the tasks below to finish your project. However, you can attempt these tasks to turn your project into a more advanced portfolio piece.

### Hyperparameter Tuning
**TODO:** Here you can perform hyperparameter tuning to increase the performance of your model. You are encouraged to 
- tune as many hyperparameters as you can to get the best performance from your model
- explain why you chose to tune those particular hyperparameters and the ranges.


In [ ]:
#TODO: Create your hyperparameter search space

In [ ]:
#TODO: Create your training estimator

In [ ]:
# TODO: Fit your estimator

In [ ]:
# TODO: Find the best hyperparameters

### Model Profiling and Debugging
**TODO:** Use model debugging and profiling to better monitor and debug your model training job.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

### Model Deploying and Querying
**TODO:** Can you deploy your model to an endpoint and then query that endpoint to get a result?

In [ ]:
# TODO: Deploy your model to an endpoint

In [ ]:
# TODO: Run an prediction on the endpoint

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done

### Cheaper Training and Cost Analysis
**TODO:** Can you perform a cost analysis of your system and then use spot instances to lessen your model training cost?

In [ ]:
# TODO: Cost Analysis

In [ ]:
# TODO: Train your model using a spot instance

### Multi-Instance Training
**TODO:** Can you train your model on multiple instances?

In [ ]:
# TODO: Train your model on Multiple Instances